In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [13]:

import sys
import os
sys.path.append('../')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [14]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_connectomes'
out_files = os.listdir(FOLDER_OUTPUT)
out_files = sorted(out_files, key=lambda x: os.path.getctime(os.path.join(FOLDER_OUTPUT, x)), reverse=True)

out_files

['comparators_drosophila_medulla_1.pkl',
 'comparators_mouse_retina_1.pkl',
 'comparators_kasthuri_graph_v4.pkl',
 'comparators_rattus.pkl',
 'comparators_c.pkl',
 'comparators_rhesus_brain_1.pkl',
 'comparators_mouse_brain_1.pkl',
 'comparators_mouse_visual.pkl',
 'comparators_rhesus_interareal.pkl',
 'comparators_rhesus_brain_2.pkl',
 'comparators_rhesus_cerebral.pkl',
 'comparators_mixed.pkl',
 'comparators_p.pkl']

In [15]:
import pickle
#pick first id from out_files
id = out_files[0]

with open(f'{FOLDER_OUTPUT}/{id}', 'rb') as f:
    comparators = pickle.load(f)


In [16]:
len(comparators)

18

In [17]:
dfs = []
for i in range(len(comparators)):
    dfs.append(comparators[i].summary_df)

dfs[0]

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,mouse_visual.cortex_1.graphml,Original,NaN,N/A,True,29,44,0.108374,0.049425,2.940887,6,-0.470896,1,29
1,mouse_visual.cortex_1.graphml,LG,1.723925,"d=1, sigma=-3.2644",True,29,81,0.199507,0.208402,2.066502,4,-0.123376,1,29
2,mouse_visual.cortex_1.graphml,ER,0.074114,0.115556,True,29,36,0.088670,0.093103,3.746032,9,0.057592,2,28
3,mouse_visual.cortex_1.graphml,WS,0.199880,20.052222,True,29,58,0.142857,0.140722,2.428571,4,-0.420017,1,29
4,mouse_visual.cortex_1.graphml,BA,1.745909,3.0,True,29,78,0.192118,0.303711,2.078818,4,-0.169849,1,29
5,mouse_visual.cortex_1.graphml,GRG,13.526541,1.0,True,29,402,0.990148,0.990608,1.009852,2,-0.081438,1,29


In [18]:
pd.concat(dfs)

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,mouse_visual.cortex_1.graphml,Original,NaN,N/A,True,29,44,0.108374,0.049425,2.940887,6.0,-0.470896,1.0,29.0
1,mouse_visual.cortex_1.graphml,LG,1.723925,"d=1, sigma=-3.2644",True,29,81,0.199507,0.208402,2.066502,4.0,-0.123376,1.0,29.0
2,mouse_visual.cortex_1.graphml,ER,0.074114,0.115556,True,29,36,0.088670,0.093103,3.746032,9.0,0.057592,2.0,28.0
3,mouse_visual.cortex_1.graphml,WS,0.199880,20.052222,True,29,58,0.142857,0.140722,2.428571,4.0,-0.420017,1.0,29.0
4,mouse_visual.cortex_1.graphml,BA,1.745909,3.0,True,29,78,0.192118,0.303711,2.078818,4.0,-0.169849,1.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,drosophila_medulla_1.graphml,LG,2.560805,"d=2, sigma=-7.1136",True,1781,79242,0.049992,0.050007,1.961010,3.0,-0.002452,1.0,1781.0
2,drosophila_medulla_1.graphml,ER,0.399940,0.01,True,1781,15700,0.009905,0.010425,2.883589,4.0,-0.010269,1.0,1781.0
3,drosophila_medulla_1.graphml,BA,0.508189,5.0,True,1781,8880,0.005602,0.025323,3.156345,5.0,-0.043973,1.0,1781.0
4,drosophila_medulla_1.graphml,WS,1.098944,20.2,True,1781,32058,0.020225,0.020075,2.453998,3.0,-0.003279,1.0,1781.0


In [19]:
# Create a dictionary to store rankings for each graph
rankings = {}

# Go through each df in dfs
for df in dfs:
    graph_name = df['graph_filename'].iloc[0]  # Get graph filename
    
    # Sort models by GIC value (excluding 'Original' which has NaN GIC)
    model_ranks = df[df['model'] != 'Original'].sort_values('gic_value')['model'].tolist()
    
    # Create rankings dictionary for this graph (1 is best since lowest GIC is best)
    rankings[graph_name] = {model: rank+1 for rank, model in enumerate(model_ranks)}

# Get unique models (excluding 'Original')
models = sorted(list(set([model for df in dfs for model in df['model'].unique() if model != 'Original'])))

# Create DataFrame with rankings
ranking_df = pd.DataFrame(rankings).T

# Ensure all models are present as columns
for model in models:
    if model not in ranking_df.columns:
        ranking_df[model] = np.nan

# Keep only the model columns in a consistent order
ranking_df = ranking_df[models]

print("Rankings by GIC value (lower is better):")
display(ranking_df)


Rankings by GIC value (lower is better):


,BA,ER,GRG,LG,WS
mouse_visual.cortex_1.graphml,4,1,5,3,2
p.pacificus_neural.synaptic_2.graphml,3,2,5,4,1
p.pacificus_neural.synaptic_1.graphml,3,2,5,4,1
mixed.species_brain_1.graphml,2,1,5,3,4
rhesus_cerebral.cortex_1.graphml,3,2,5,1,4
rhesus_brain_2.graphml,3,1,5,2,4
rhesus_interareal.cortical.network_2.graphml,3,1,2,4,5
mouse_visual.cortex_2.graphml,5,2,3,4,1
mouse_brain_1.graphml,4,2,3,1,5
rhesus_brain_1.graphml,1,2,5,3,4


In [20]:
ranking_df.mean(axis=0)

BA     2.833333
ER     1.888889
GRG    4.388889
LG     2.666667
WS     3.222222
dtype: float64